# How To Issue Natural Language Queries To An Existine Pinecone Index via LlamaIndex

This past weekend, I had the privilege of giving a talk & judging the finalists at [AGI House](https://agihouse.ai/)'s [Build / Break / Test](https://partiful.com/e/QKjcXJOCb2IHiuAVX9L6) hackathon.

Throughtout the hackathon, multiple engineers came up to me asking for a concise way of querying an existing Pinecone index, especially one that was [namespace-ed](https://docs.pinecone.io/docs/namespaces). While it's not currently possible to use [LlamaIndex's popular `as_query_engine()` method](https://gpt-index.readthedocs.io/en/latest/core_modules/query_modules/query_engine/root.html) to query an existing Pinecone index with namespaces, there is still a beautiful way to get the job done.

In [1]:
# Make sure you have the libraries below installed

import os

from llama_index import VectorStoreIndex
from llama_index import ServiceContext, OpenAIEmbedding
from llama_index.indices.vector_store import VectorIndexRetriever
from llama_index.llms import OpenAI
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.vector_stores import PineconeVectorStore

In [2]:
# You'll need to set your Pinecone and OpenAI environment variables.
# You can find your Pinecone API key and environment name in the UI at app.pinecone.io, once you've made an account.

os.environ['PINECONE_ENVIRONMENT'] = ''
os.environ['PINECONE_API_KEY'] = ''
os.environ['OPENAI_API_KEY'] = ''

In [3]:
# Initialize your Pinecone vector store

vector_store = PineconeVectorStore(api_key=os.getenv('PINECONE_API_KEY'),
                                   environment=os.getenv('PINECONE_ENVIRONMENT'),
                                   index_name='namespaces-demo',
                                   namespace="en",
                                   text_key='text_content')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
# Build your Service Context object

llm = OpenAI(model='ada', api_key=os.getenv('OPENAI_API_KEY'))
embed_model = OpenAIEmbedding(api_key=os.getenv('OPENAI_API_KEY'))

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)

In [ ]:
# Build your Index object

index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)


In [ ]:
# Build your Retriever object

vector_index_retriever = VectorIndexRetriever(index)


In [ ]:
# Build your Query Engine (this is where you might use the `as_query_engine()` method in other use cases)

query_engine = RetrieverQueryEngine(vector_index_retriever)
    

In [ ]:
# Query your engine!

response = query_engine.query("What is a dog?")
